# RouterChains

As Router Chains são cadeias mais avançadas que permitem roteamento de prompts com base na entrada do usuário. Elas são úteis quando você precisa lidar com diferentes tipos de perguntas ou problemas.

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain.llms import OpenAI

llm = OpenAI(model='gpt-3.5-turbo-instruct')

C:\Users\annaf\AppData\Local\Temp\ipykernel_10496\1267886439.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(model='gpt-3.5-turbo-instruct')


In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [4]:
fis_template = ChatPromptTemplate.from_template("""Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: {input}""")

mat_template = ChatPromptTemplate.from_template("""Você é um matemático muito bom.
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor
problemas difíceis em suas partes componentes, 
responder às partes componentes e depois juntá-las
para responder à pergunta mais ampla.

Aqui está uma pergunta: {input}""")

hist_template = ChatPromptTemplate.from_template("""Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: {input}""")


In [5]:
prompt_infos = [
    {
        'name': 'Física',
        'description': 'Ideal para responder perguntas sobre Física',
        'prompt_template': fis_template,
    },
    {
        'name': 'Matemática',	
        'description': 'Ideal para responder perguntas sobre Matemática',
        'prompt_template': mat_template,
    },
    {
        'name': 'História',	
        'description': 'Ideal para responder perguntas sobre História',
        'prompt_template': hist_template,
    }
]

In [7]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [8]:
chains_destino = {}

for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)
    chains_destino[info['name']] = chain
    
chains_destino

C:\Users\annaf\AppData\Local\Temp\ipykernel_10496\1188194747.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)


{'Física': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='Você é um professor de física muito inteligente.\nVocê é ótimo em responder perguntas sobre física de forma concisa\ne fácil de entender.\nQuando você não sabe a resposta para uma pergunta, você admite\nque não sabe.\n\nAqui está uma pergunta: {input}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BF582DEF60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BF582DFE60>, root_client=<openai.OpenAI object at 0x000001BF44B30740>, root_async_client=<openai.AsyncOpenAI object at 0x000001BF582DEF00>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={})

In [10]:
destinos = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]
destinos_str = '\n'.join(destinos)
print(destinos_str)

Física: Ideal para responder perguntas sobre Física
Matemática: Ideal para responder perguntas sobre Matemática
História: Ideal para responder perguntas sobre História


In [12]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinos_str
)

print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Física: Ideal para responder perguntas sobre Física
Matemática: Ideal para responder perguntas sobre 

In [13]:
router_template = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [14]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True
)

C:\Users\annaf\AppData\Local\Temp\ipykernel_10496\2250053747.py:4: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [15]:
chain.invoke({'input': 'O que é um buraco negro?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Física: {'input': 'O que é um buraco negro?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: O que é um buraco negro?

> Finished chain.

> Finished chain.


{'input': 'O que é um buraco negro?',
 'text': 'Um buraco negro é uma região do espaço-tempo onde a gravidade é tão intensa que nada, nem mesmo a luz, consegue escapar de seu campo gravitacional. Ele é formado a partir do colapso de uma estrela massiva no final de sua vida. A massa é concentrada em um ponto de densidade infinita chamado singularidade, cercado por uma região de onde nada pode escapar, chamada de horizonte de eventos. Os buracos negros são objetos extremamente densos e exibem propriedades únicas devido à intensidade de sua gravidade.'}

In [16]:
chain.invoke({'input': 'O que é uma equação quadrática?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Matemática: {'input': 'O que é uma equação quadrática?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um matemático muito bom.
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor
problemas difíceis em suas partes componentes, 
responder às partes componentes e depois juntá-las
para responder à pergunta mais ampla.

Aqui está uma pergunta: O que é uma equação quadrática?

> Finished chain.

> Finished chain.


{'input': 'O que é uma equação quadrática?',
 'text': 'Uma equação quadrática é uma equação polinomial de segunda ordem, ou seja, uma equação na qual a incógnita é elevada ao quadrado. Geralmente é escrita na forma ax^2 + bx + c = 0, onde a, b e c são constantes e x é a incógnita. A solução de uma equação quadrática pode ser encontrada utilizando a fórmula de Bhaskara ou completando o quadrado, dependendo do caso. As soluções de uma equação quadrática podem ser reais ou complexas.'}

In [17]:
chain.invoke({'input': 'Quando ocorreu a Revolução Industrial?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
História: {'input': 'Quando ocorreu a Revolução Industrial?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: Quando ocorreu a Revolução Industrial?

> Finished chain.

> Finished chain.


{'input': 'Quando ocorreu a Revolução Industrial?',
 'text': 'A Revolução Industrial ocorreu principalmente durante o século XVIII e XIX, com início na Inglaterra por volta do final do século XVIII. Este período foi marcado por avanços significativos na tecnologia, produção industrial, urbanização e mudanças sociais, que tiveram um impacto profundo na economia, política e sociedade em todo o mundo.'}